In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from helpers.least_squares import *
from helpers.gradient_descent import * 
import pandas as pd
from helpers.costs import *

%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [6]:
from helpers.proj1_helpers import *
DATA_TRAIN_PATH = '/Users/mac/Documents/GitHub/ml-project-1-aaa_project1/data/train.csv'
# TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [7]:
#Group the data into three sets, depending on whether their PRI_jet_num is {0,1,(2|3)} and return the created sets => also return the corresponding y's
def groupy_by_jet_num(x,y):
    #create masks to extract each one of the subsets
    mask0 = x[:,22] == 0
    mask1 = x[:,22] == 1
    mask2 = x[:,22] == 2
    mask3 = x[:,22] == 3
    mask2_3 = np.logical_or(mask2,mask3)
    
    #extract the elements from each subset and return the subsets
    jet_0 = x[mask0,:]
    jet_1 = x[mask1,:]
    jet_2_3 = x[mask2_3,:]
    
    #extract the corresponding labels
    label_0 = y[mask0]
    label_1 = y[mask1]
    label_2_3 =  y[mask2_3]
    return jet_0, label_0, jet_1, label_1, jet_2_3, label_2_3

In [8]:
#For each one of the three sets, filter out the columns (features) that have only invalid (-999) values
def remove_invalid_features(jet_0,jet_1,jet_2_3):
    #we create a mask of the columns that are invalid for each subset
    invalid_jet_1 = [4,5,6,12,22,23,24,25,26,27,28,29]
    invalid_jet_2 = [4,5,6,12,22,26,27,28]
    
    #we remove the invalid elements from each subset
    jet_0 = np.delete(jet_0,invalid_jet_1,axis=1)
    jet_1 = np.delete(jet_1,invalid_jet_2,axis=1)

    return jet_0,jet_1,jet_2_3

In [9]:
def remove_outliers(x):
    '''go through every column and calculate it's mean'''
    nbColumns = x.shape[1]
    for i in range(nbColumns):
        #we calculate the median of the current column after discarding the -999 values (they should not be in the median)
        median = np.median(x[:,i][x[:,i]!= -999])
        
        #we find the indices of the elements with value -999 in our current column
        indices = x[:,i] == -999
        
        #we replace the element at the found indices by the median of the current column
        x[:,i][indices] = median
    
    #return x

In [19]:
def compute_gradient(y, tx, w):
    """Computes the gradient"""
    error = y - tx@w
    return (tx.T@error)/(y.shape[0])

def compute_loss(y, tx, w):
    """Computes the loss by MSE"""
    error = y - tx@w
    N= y.shape[0]
    return 1/(2) * np.mean(error**2)


def gradient_descent(y, tx, initial_w, max_iters, gamma):
    """Performs the gradient descents algorithm and returns the last weights and loss value"""
    # initialize the weights
    w = initial_w
    
    for n_iter in range(max_iters):
        
        # compute loss and gradient
        loss=compute_loss(y, tx, w)
        
        gradient=compute_gradient(y, tx, w)
        print(n_iter)
        print(w)
        #print(loss)
        
        # update the weights 
        w = w - gamma * gradient
        #if n_iter % 200 == 0:
            #print("Gradient Descent({bi}/{ti}): loss={l}".format(bi=n_iter, ti=max_iters - 1, l=loss))

    return w, loss

In [28]:
def compute_squared_loss(y, tx, w):
    """Calculate the loss.

    You can calculate the loss using mse or mae.
    """
    # w use mse
    # in the theoretical part of the session we have derived that L(w) = 1/2N e'*e
    N = y.shape[0]
    e = y- np.dot(tx,w)
    
    return (1/2) * np.mean(e**2) 

def compute_log_likelihood(y,tx,w):
    """compute the loss: negative log likelihood."""
    product = np.dot(tx,w)
    exp = np.exp(product)
    log = np.log(exp+1)
    one = y * product
    diff = log - one
    return sum(diff)

In [29]:
def compute_gradient(y, tx, w):
    """Compute the gradient."""
    e = y - np.dot(tx,w)
    N = y.shape[0]
    return (-1/N)* np.dot(np.transpose(tx),e)


def gradient_descent(y, tx, initial_w, max_iters, gamma):
    """Gradient descent algorithm."""
    w = initial_w
    for n_iter in range(max_iters):
        
        grad = compute_gradient(y,tx,w)
        
        w = w - gamma * grad 
     
        # store w and loss

    loss = compute_squared_loss(y,tx,w)
  
    return w, loss

In [30]:

def compute_stoch_gradient(y, tx, w):
        e = y - np.dot(tx,w)
        N = y.shape[0]
        return (-1/N)* np.dot(np.transpose(tx),e)


def stochastic_gradient_descent(y, tx, initial_w, max_iters, gamma, batch_size = 1 ):
        w = initial_w
        for n_iter in range(max_iters):
            for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size):
                grad = compute_stoch_gradient(minibatch_y,minibatch_tx,w)
                
                w = w - gamma * grad
                
        loss = compute_squared_loss(minibatch_y,minibatch_tx,w)        
        return w, loss

def compute_stoch_gradient(y, tx, w):
        e = y - np.dot(tx,w)
        N = y.shape[0]
        return (-1/N)* np.dot(np.transpose(tx),e)


def stochastic_gradient_descent( y, tx, initial_w, max_iters, gamma, batch_size = 1 ):
        w = initial_w

        
        for n_iter in range(max_iters):
            for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size):
                grad = compute_stoch_gradient(minibatch_y,minibatch_tx,w)
                print(grad )
                w = w - gamma * grad
                print(w)
        #loss = compute_squared_loss(minibatch_y,minibatch_tx,w)    
        loss = compute_loss_mse(minibatch_y,minibatch_tx,w)    

        
        return w, loss

In [31]:
def ridge(y, tx, lambda_):
    """implement ridge regression."""
    x_t = np.transpose(tx)
    N = tx.shape[0]
    K = tx.shape[1]
    Id = 2 * tx.shape[0] * lambda_ * np.identity(tx.shape[1])
    one = np.dot(x_t,tx) + Id
    two = np.dot(x_t,y)
    w = np.linalg.solve(one,two)
    return w, compute_squared_loss(y,tx,w)

In [32]:
def sigmoid(t):
    
    if (np.all(t)<-100):
        t=-100
    if (np.all(t)>100):
        t=100
    val=1/(1 + np.exp(-t))
    print(np.all(val)>0)
    print(val)
    return val


def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    prod = np.dot(tx,w)
    x_t = np.transpose(tx)
    return np.dot(x_t, sigmoid(prod)-y)

def loss_logistic(data, labels, w):
    print(data.shape)
    print(w.shape)
    return -np.sum(labels * np.log(sigmoid(data @ w)) + (1 - labels) * np.log(1 - sigmoid(data @ w)))

def compute_loss_log_reg(tx,y,w):
    print(tx.shape)
    print(y.shape)
    print(w.shape)
    one = np.log(sigmoid(tx@w))
    two = np.log(1-sigmoid(tx@w))
    print(one, two)
    print("after")
    return - np.sum(y*np.log(sigmoid(tx@w))+(1-y)*np.log(1-sigmoid(tx@w)))


def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descent using logistic regression.
    Return the loss and the updated w.
    """
    print('Start')
    loss = compute_loss_log_reg(tx, y,w)
    print('Loss')
    grad = calculate_gradient(y,tx,w)
    print('Grad')
    w = w - gamma*grad
  
    return loss, w


def logistic_reg(y, tx,initial_w,max_iter,gamma):
    
    tx = np.c_[np.ones((y.shape[0], 1)), tx]
    w = np.zeros((tx.shape[1], 1))
    w= initial_w
    loss = 0

    for iter in range(max_iter):

        loss, w = learning_by_gradient_descent(y, tx, w, gamma)
        print(iter)

    return w,loss

# Test des fonctions and stufffff


In [33]:
tX,_,_= standardize(tX)

In [35]:
#LS with Gradient Descent Part.
#jet_0,label_0,jet_1,label_1,jet_2_3, label_2_3= groupy_by_jet_num(tX,y)
    
#remove_outliers(tX)
init_weights = np.ones((30,))
print(y.shape)
print(tX.shape)

weights,loss =  gradient_descent(y, tX,init_weights , 1500, .01)

print(weights)
print("{:e}".format(loss))


(250000,)
(250000, 30)
[ 0.21438946 -0.25020933 -0.18675247  0.13013038  0.01426429  0.23598701
  0.0102705   0.19477731  0.08424929 -0.38594631  0.19345338  0.20914762
  0.01846474  0.20884206  0.20514039  0.20509995 -0.02520601  0.20515137
  0.20523347  0.07841806  0.20555975 -0.56623307  0.1942683  -0.02785991
  0.07418741  0.07427074 -0.16647721  0.01935423  0.01915115 -0.15891145]
4.185785e-01


In [34]:
#Least Squares
weight,loss= least_squares(y,tX)
print(compute_loss(y,tX,weight))

0.33944681722221465


#LS with StochaGradient Descent Part.

remove_outliers(tX)
init_weights = np.ones((30,))
print(y.shape)
print(tX.shape)

weights,loss =  stochastic_gradient_descent(y, tX,init_weights , 500, 0.01, batch_size=1)

print(weights)
print(loss)

#Ridge_Reg
#print(y.shape)
#print(tX.shape)

weights, loss = ridge(y, tX, 0.05)
print("{:e}".format(np.sqrt(2*loss)))

In [ ]:
init_weights = np.random.normal(0., 0.1, [tX.shape[1]+1,1])
print(tX.shape)
tX,_,_=standardize(tX)
print(tX.shape)
print('Standardisé')
weight, loss= logistic_reg(y, tX, init_weights, 2, 0.01)
print(loss)

(250000, 30)
(250000, 30)
Standardisé
Start
(250000, 31)
(250000,)
(31, 1)
True
[[0.59114628]
 [0.38927518]
 [0.34088096]
 ...
 [0.38857514]
 [0.3395192 ]
 [0.28953912]]
True
[[0.59114628]
 [0.38927518]
 [0.34088096]
 ...
 [0.38857514]
 [0.3395192 ]
 [0.28953912]]
[[-0.52569177]
 [-0.94346878]
 [-1.07622194]
 ...
 [-0.94526871]
 [-1.08022478]
 [-1.23946486]] [[-0.89439785]
 [-0.4931088 ]
 [-0.41685113]
 ...
 [-0.49196322]
 [-0.41478723]
 [-0.34184139]]
after
True
[[0.59114628]
 [0.38927518]
 [0.34088096]
 ...
 [0.38857514]
 [0.3395192 ]
 [0.28953912]]


## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '/Users/abdeslamguessous/Documents/MA1/ML/ML_course/projects/ml-project-1-aaa_project1/data/test.csv' # TODO: download train data and supply path here 
y_test, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
                                     
                                     

In [ ]:
OUTPUT_PATH = '/Users/abdeslamguessous/Documents/MA1/ML/ML_course/projects/ml-project-1-aaa_project1/data/output_files/output.csv' 
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

def success_rate(predicted_labels,true_labels):
    """calculate the success rate of our predictions """
    success_rate = 1 - (np.count_nonzero(predicted_labels - true_labels)/len(predicted_labels))
    return success_rate

1-success_rate(y_pred, y_test)

print(y_test)